# HSE Summer School NLP&DA

![](https://www.hse.ru/data/2014/06/25/1309038576/logo_hse_cmyk_e.jpg)

Учебный проект по теме "Тональность отношений субъектов (именованных сущностей)", предполагающий разработку участниками школы под руководством тьюторов программных средств, решающих задачу определения мнения сторон о различных событиях, освещаемых в новостях

Нужен алгоритм, который по выделенным сущностям может найти пары этих сущностей в тексте и вырезать соответствующие куски между ними + несколько слов справа-слева. Возможно, имеет смысл идти от N-ой выделенной сущности до N+2, и включать всё до неё. 

Чем может помочь синтаксический анализ?

Чем поможет морфологический анализ?

In [1]:
import sys 
sys.path.append("/Users/dmitrys/anaconda2/lib/python2.7/site-packages/")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymorphy2
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

import string
%matplotlib inline

## Utility functions

In [2]:
def loadAnswer(number):
    with open("Texts/art{}.opin.txt".format(number)) as f:
        d = f.read()
    return d

def loadText(number):
    with open("Texts/art{}.txt".format(number)) as f:
        d = f.read()
    return d

def transformAnnotation(number):
    """
    Given number loads txt file with annotation 
    Returns DataFrame with transformed annotation
    """
    with open("Texts/art{}.ann".format(number)) as f:
        d = f.read()

    d = d.split("\n")

    for i in range(len(d)):
        d[i] = d[i].split("\t")

    d = pd.DataFrame(d)
    d.drop([0], axis=1, inplace=True)
    d = pd.concat([d, pd.DataFrame(d[1].apply(lambda x: x.split()).tolist())], axis=1)
    d.columns = ["to_delete", "entity", "entity_car", "pos_1", "pos_2"]
    d.drop(["to_delete"], axis=1, inplace=True)
    d["entity"] = d["entity"].apply(lambda x: x.strip("\r"))
    d["entity"] = d["entity"].apply(lambda x: x.decode("utf8"))
    
    return d

def transformAnswer(number):
    """
    Given number loads txt file with answer 
    Returns DataFrame with transformed answer
    """
    answ = loadAnswer(number)
    answ = answ.split("\n")

    for i in range(len(answ)):
        answ[i] = answ[i].split(",")

    answ = pd.DataFrame(answ)
    answ.columns = ["entity_1", "entity_2", "attitude", "time"]
    answ.dropna(inplace=True)
    answ.time = answ.time.apply(lambda x: x.strip("\r"))
    
    return answ

def loadRuSentiLex():
    """
    Loads the RuSentiLex 2017 dictionary
    Returns data frame with ["word", "tag", "word_lemmatized", "tone", "certainty"]
    """
    with open("RuSentiLex2017_revised_2.txt") as f:
        Rusentilex = f.read().decode("cp1251").encode("utf8")
        Rusentilex = Rusentilex[1510:]

    Rusentilex = Rusentilex.split("\n")
    for i, item in enumerate(Rusentilex):
        Rusentilex[i] = item.split(",")

    for i in Rusentilex:
        if len(i) < 5:
            Rusentilex.remove(i)

    Rusentilex = pd.DataFrame(Rusentilex)
    Rusentilex.drop([5, 6, 7], axis=1, inplace=True)
    Rusentilex.columns = ["word", "tag", "word_lemmatized", "tone", "certainty"]
    Rusentilex["certainty"] = Rusentilex["certainty"].apply(lambda x: x.strip('\r'))
    
    
    Rusentilex["tone"] = Rusentilex["tone"].apply(lambda x: x.strip(' '))
    Rusentilex["certainty"] = Rusentilex["certainty"].apply(lambda x: x.strip(' '))
    
    return Rusentilex

def cleanString(myString):
    return myString.translate(None, string.punctuation).decode('utf-8')

![](http://cathyreisenwitz.com/wp-content/uploads/2016/01/no.jpg)

Словарь РуСентиЛекс

Структура: 
- 1 слово или словосочетание,
- 2 Часть речи или синтаксический тип группы,
- 3 слово или словосочетание в лемматизированной форме, 
- 4 Тональность: позитивная (positive), негативная(negative), нейтральная (neutral) или неопределеная оценка, зависит от контекста (positive/negative),
- 5 Источник: оценка (opinion), чувство (feeling), факт (fact),
- 6 Если тональность отличается для разных значений многозначного слова, то перечисляются все значения слова по тезаурусу РуТез и дается отсылка на сооветствующее понятие - имя понятия в кавычках.

In [3]:
Rusentilex = loadRuSentiLex()

In [4]:
Rusentilex.head()

,word,tag,word_lemmatized,tone,certainty
0,аборт,Noun,аборт,negative,fact
1,абортивный,Adj,абортивный,negative,fact
2,абракадабра,Noun,абракадабра,negative,opinion
3,абсурд,Noun,абсурд,negative,opinion
4,абсурдность,Noun,абсурдность,negative,opinion


In [5]:
Rusentilex.certainty.value_counts()

opinion             9269
fact                5297
feeling             1536
operator              41
negative               2
positive               2
fact В ТЕЗ empty       1
fact В ТЕЗ EMPTY       1
opinegativenion        1
Name: certainty, dtype: int64

In [6]:
def getSentimentCertainity(word):
    tone, certainty =  Rusentilex[["tone", "certainty"]][Rusentilex.word.isin([word])].values[0]
    return tone, certainty

In [7]:
getSentimentCertainity("аборт")

('negative', 'fact')

In [8]:
entities = transformAnnotation(1)
a = transformAnswer(1) 

In [9]:
# надо идти по всем сущностям с правилами остановки

In [10]:
t = loadText(1)

In [11]:
print(t[:1500])

{Author, Unknown} 

{Author, Unknown} СМИ Ирана: Финляндия не хочет вступать в НАТО из страха перед Россией?

{Author, Unknown} inosmi.ru

{Author, Unknown} 5-го, 2016


{Author, Unknown} Зачем американцам понадобилось перебросить 250 своих военных на северо-восток Сирии, к окрестностям города Румийлан, разбирался обозреватель Khorasan (
{Author, Unknown} 30.04) Алиреза Резахах (Alireza Rezakhah). 
{Author, Unknown} Согласно заявлениям из Пентагона, цель США — способствовать освобождению от боевиков ИГИЛ стратегически важного города Ракка. 
{Author, Unknown} Однако в Дамаске действия Вашингтона резко осудили и назвали покушением на территориальный суверенитет Сирии. 
{Author, Unknown} Примерно в аналогичном ключе высказалась и Москва, где «несогласованные с Дамаском» шаги Белого дома назвали фактором, вызывающим озабоченность.


{Author, Unknown} Почему же и в Москве и Дамаске, которые также ведут борьбу прот


In [12]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [13]:
p = morph.parse(u"аборт")[0]
p.tag.case

u'nomn'

In [14]:
entities[~entities.entity.isin(["Unknown", "Author"])].head(10)

,entity,entity_car,pos_1,pos_2
4,СМИ,ORG,39,42
5,Ирана,LOC,43,48
6,Финляндия,GEOPOLIT,50,59
7,НАТО,ORG,80,84
8,Россией,GEOPOLIT,101,108
11,ru,ORG,137,139
16,Сирии,LOC,271,276
17,Румийлан,LOC,300,308
18,Khorasan,ORG,334,342
21,Алиреза Резахах (Alireza Rezakhah,PER,370,403


In [15]:
def Text_to_dict(number):
    t = loadText(number)
    text_dict = t.split("\n\n")
    text_dict = {i:parag for i, parag in enumerate(text_dict)}
    sentence_dict = {i:{} for i in range(len(text_dict))}

    for key, paragraph in text_dict.iteritems():
        paragraph = paragraph.split("{Author, Unknown}")
        for sent_numbet, sentence in enumerate(paragraph):

            sentence = cleanString(sentence)
            if len(sentence) != 0:
                sentence_dict[key][sent_numbet] = sentence
                #tknzr.tokenize()
    return sentence_dict

In [16]:
# for key, value in t.iteritems():
#     print("paragraph number", key)
#     for another_key, another_value in value.iteritems():
        
#         print("sentence_number", another_key)
#         for word in another_value:
#             print word

In [17]:
text = Text_to_dict(1)

In [18]:
entities = entities[~entities.entity.isin(["Unknown", "Author"])].entity.reset_index(drop=True)

In [19]:
def getEntityPositions(text, entities):
    ENTITIES = pd.DataFrame(columns=["entity", "paragraph", "sentence", "loc_start", "loc_end"])
    
    for paragraph in text:
        for sentence in text[paragraph]:
            #for word in text[paragraph][sentence]:
            for entity in list(entities):
                if entity in text[paragraph][sentence]:
                    
                    loc_start = text[paragraph][sentence].find(entity)
                    loc_end = loc_start + len(entity)
                    
                    to_append =   {"entity":entity, 
                                   "paragraph":paragraph, 
                                   "sentence":sentence,
                                   "loc_start":loc_start,
                                   "loc_end":loc_end}
                    
                    ENTITIES = ENTITIES.append(to_append, ignore_index = True)
    return ENTITIES.drop_duplicates()

In [20]:
ENTITIES = getEntityPositions(text, entities)

In [21]:
len(ENTITIES.entity.unique())

94

In [22]:
for entity in entities.unique():
    if entity not in ENTITIES.entity.unique():
        print entity

Алиреза Резахах (Alireza Rezakhah
Аль-Джазира
Али Алеми (Ali Alemi
Хоссаму Аль-Саеду
премьер-министр Италии


In [26]:
entities.head()

0          СМИ
1        Ирана
2    Финляндия
3         НАТО
4      Россией
Name: entity, dtype: object

In [27]:
ENTITIES.head()

,entity,paragraph,sentence,loc_start,loc_end
0,СМИ,1,1,1,4
1,Ирана,1,1,5,10
2,Финляндия,1,1,11,20
3,НАТО,1,1,41,45
4,Россией,1,1,62,69


In [20]:
from pymystem3 import Mystem

m = Mystem()
def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

In [15]:
for i in range(10):
    print(lemmatize(t[i]))

{Author,
Unknown}
{Author,
Unknown}
россия
и
кратковременный
успех
в
сирия
